In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
training = spark.read.format("libsvm").load("/FileStore/tables/sample_linear_regression_data-1.txt")
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [0]:
lr = LinearRegression(featuresCol='features', labelCol='label',predictionCol='prediction')

In [0]:
lrModel = lr.fit(training)

In [0]:
lrModel.coefficients

Out[7]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
lrModel.intercept

Out[8]: 0.14228558260358093

In [0]:
training_summary = lrModel.summary

In [0]:
training_summary.rootMeanSquaredError

Out[10]: 10.16309157133015

In [0]:
ad = spark.read.format("libsvm").load("/FileStore/tables/sample_linear_regression_data-1.txt")

In [0]:
train, test=ad.randomSplit([.7,.3])

In [0]:
train.describe().show()

+-------+--------------------+
summary| label|
+-------+--------------------+
 count| 337|
 mean|-0.11379711900698904|
 stddev| 10.3100218298228|
 min| -28.571478869743427|
 max| 27.78383192005107|
+-------+--------------------+

In [0]:
test.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 164|
 mean| 1.0186032257624653|
 stddev| 10.323676944776482|
 min|-26.736207182601724|
 max| 26.903524792043335|
+-------+-------------------+

In [0]:
cor = lr.fit(train)

In [0]:
res = cor.evaluate(test)

In [0]:
res.residuals.show()

+-------------------+
 residuals|
+-------------------+
-22.300612639434057|
 -18.63812710665806|
-19.403293769514477|
 -20.8074713302029|
-15.789841665748398|
-15.679863421798384|
-15.296801931624444|
 -18.23580534604165|
 -15.35392158389718|
-15.011406341353235|
-16.940328500832788|
-12.884532968017632|
-16.927036128719745|
-12.061564615774232|
-14.242506373917605|
-10.137633271301265|
-10.277149238587807|
-13.054226523196926|
 -13.42662981417969|
 -8.708949143698344|
+-------------------+
only showing top 20 rows

In [0]:
res.rootMeanSquaredError

Out[24]: 10.373882056902513

In [0]:
unlabeled = test.select('features')

In [0]:
unlabeled.show()

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

In [0]:
preds = cor.transform(unlabeled)
preds.show()

+--------------------+--------------------+
 features| prediction|
+--------------------+--------------------+
(10,[0,1,2,3,4,5,...| -4.435594543167669|
(10,[0,1,2,3,4,5,...| -4.19933331026128|
(10,[0,1,2,3,4,5,...| -0.481267004758949|
(10,[0,1,2,3,4,5,...| 1.9615488573043192|
(10,[0,1,2,3,4,5,...| -2.013784522916119|
(10,[0,1,2,3,4,5,...| -1.8143369350849587|
(10,[0,1,2,3,4,5,...| -2.029918801051505|
(10,[0,1,2,3,4,5,...| 1.1704057201656353|
(10,[0,1,2,3,4,5,...| -0.5975909818973943|
(10,[0,1,2,3,4,5,...|-0.42597845207798346|
(10,[0,1,2,3,4,5,...| 1.5807836210001094|
(10,[0,1,2,3,4,5,...| -2.464338187361621|
(10,[0,1,2,3,4,5,...| 3.1545945670168734|
(10,[0,1,2,3,4,5,...| -1.0917709905912978|
(10,[0,1,2,3,4,5,...| 1.751064296371192|
(10,[0,1,2,3,4,5,...| -1.9927199409866638|
(10,[0,1,2,3,4,5,...| -1.817202039947451|
(10,[0,1,2,3,4,5,...| 1.1760590232300854|
(10,[0,1,2,3,4,5,...| 1.9948274486392301|
(10,[0,1,2,3,4,5,...| -2.3303986645554837|
+--------------------+--------------------+
only showing top 20 rows